In [117]:
import os
from pathlib import Path
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# from astropy.coordinates import SkyCoord
# import astropy.units as u

# from astroquery.gaia import Gaia
# Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
# Gaia.ROW_LIMIT = 5

from astroquery.simbad import Simbad
simbad = Simbad()
simbad.add_votable_fields('otype')

In [86]:
def get_coord(tics, sec = 6):
    data_dir = '/mnt/disks/DATA_DIR/tess-goddard-lcs/'

    ref = pd.read_csv(data_dir + f'sector{sec}lookup.csv').set_index('TIC_ID')
    subref = ref.loc[tics]
    files = data_dir + subref.Filename.values
    
    ra_dict = subref.RA.to_dict()
    dec_dict = subref.Dec.to_dict()
    
    coord_dict = {**ra_dict, **dec_dict}
    for key, value in coord_dict.items():
        if key in ra_dict and key in dec_dict:
            coord_dict[key] = [ra_dict[key], value]

    return coord_dict

In [3]:
def get_tess():
    tess_dir = os.listdir('tess_figs')
    tess_dir.remove('.ipynb_checkpoints')

    tics = []
    for i in tess_dir:
        tic = i.split(' ')[1]
        tic = tic.split('.')[0]
        
        tics.append(int(tic))
    
    coord = get_coord(tics)

    return coord

In [4]:
def query_gaia(ra, dec):
    coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
    radius = u.Quantity(1.0, u.arcsec)

    job = Gaia.cone_search_async(coord, radius)
    results = job.get_results()

    gaia = int(str(results[0][1])[8:])
    print('\nGAIA: ' + str(gaia))

    query_str = "SELECT source.* " + "FROM gaiadr3.gaia_source as source " + "WHERE source.source_id='" + str(gaia) + "'"
    job = Gaia.launch_job(query_str)
    results = job.get_results()

    if results['has_epoch_photometry'][0]:
        print('GAIA: Has epoch photometry.')
        return gaia, True
    else:
        print('GAIA: Does not have epoch photometry.')
        return gaia, False

In [5]:
def get_gaia(tics):
    gaia_ids = {}

    for tic in tics:
        ra, dec = tics[tic]
        try:
            gaia_id = query_gaia(ra, dec)
            gaia_ids[tic] = gaia_id
        except IndexError:
            print(tic, ' not found in Gaia.')
            gaia_ids[tic] = None
            
    return gaia_ids

In [6]:
def get_gaia_lc(gaia_id):
    datalink = Gaia.load_data(ids=gaia_id, data_release = 'Gaia DR3', data_structure = 'COMBINED', retrieval_type = 'EPOCH_PHOTOMETRY', valid_data=True)

    dl_key = [i for i in datalink.keys()][0]
    print(f'The following Datalink product has been downloaded: * {dl_key}')

    epoch = datalink[dl_key][0].to_table().to_pandas()
    # g = epoch[epoch['band'] == 'G']
    bp = epoch[epoch['band'] == 'BP']
    rp = epoch[epoch['band'] == 'RP']
    
    return bp, rp

In [31]:
def graph_gaia(data):
    xs = []
    ys = []
    
    for index, row in data.iterrows():
        plt.clf()
        
        bp, rp = get_gaia_lc(row['GAIA_ID'])
        
        bp_rp = pd.merge_asof(bp, rp, on='time', tolerance=1, direction='nearest').dropna()  # throw out matches > 1 day
        bp_sub_rp = bp_rp['mag_x'] - bp_rp['mag_y'].values
        rp = bp_rp['mag_y']

        fit = np.polyfit(bp_sub_rp, rp, 1)
        x = np.linspace(bp_sub_rp.min(),bp_sub_rp.max(),100)
        y = fit[0]*x+fit[1]
        fit_eq = 'y=' + '{:.2f}'.format(fit[0]) + 'x+' + '{:.2f}'.format(fit[1])

        plt.scatter(bp_sub_rp, rp)
        plt.plot(x, y, label=fit_eq)

        plt.ylabel('RP Magnitude')
        plt.xlabel('BP - RP Magnitude')

        plt.gca().invert_yaxis()

        plt.legend(loc='upper left')

        fig_path = 'gaia_figs/' + 'TIC ' + str(index) + '.png'
        plt.savefig(fig_path, dpi=150, bbox_inches='tight')
        
        xs.append(fit[1])
        ys.append(fit[0])
    
    return xs, ys

In [121]:
def query_simbad(tic):
    obj = 'TIC ' + str(tic)
    result_table = simbad.query_object(obj)
    simbad_id = result_table['MAIN_ID'][0]
    simbad_otype = result_table['OTYPE'][0]
    return simbad_id, simbad_otype

In [132]:
def get_simbad(data):
    simbad_dict = {}
    i = 1
    for tic in data.index:
        print(i)
        try:
            simbad_id, simbad_otype = query_simbad(tic)
            simbad_dict[tic] = [simbad_id, simbad_otype]
        except TypeError or UserWarning:
            pass
        i += 1
    return simbad_dict

In [32]:
# tic_coords = get_tess()
# gaia = get_gaia(tic_coords)
# xs, ys = graph_gaia(df[gaia_phot_mask])
# result = get_simbad(df)

In [81]:
df = pd.read_csv('tess_gaia.csv').set_index('TIC_ID')

In [148]:
df['TESS_LC'] = np.nan

In [150]:
df['GAIA_EXT'] = np.nan

In [151]:
df

,RA,DEC,GAIA_ID,GAIA_PHOT,GAIA_X,GAIA_Y,SIMBAD_ID,SIMBAD_OTYPE,TESS_LC,GAIA_EXT
TIC_ID,,,,,,,,,,
50794137,83.402290,-4.206550,3209848692045675520,False,NaN,NaN,V* V724 Ori,OrionV*,NaN,NaN
283201557,104.077322,-5.634002,3101177184707222528,True,9.375555,2.369232,NaN,NaN,NaN,NaN
11286067,84.644126,-2.734472,3216109002016118784,False,NaN,NaN,2MASS J05383459-0244040,EmLine*,NaN,NaN
232283749,92.144299,5.758957,3318898701967199232,False,NaN,NaN,NaN,NaN,NaN,NaN
332974488,85.166891,-6.530535,3016239537801576960,False,NaN,NaN,V* V1189 Ori,OrionV*,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
43318487,97.565349,-0.766659,3119156055246902784,True,11.924636,-0.045436,EM* GGA 391,EmLine*,NaN,NaN
269117420,80.964482,0.616846,3221856630330637312,False,NaN,NaN,2MASS J05235147+0037007,YSO,NaN,NaN
176469665,86.807714,0.004785,3219144306943363072,False,NaN,NaN,EM* LkHA 313,TTauri*,NaN,NaN
